# Sparklehorse - Der SQL-Chatbot der Magpie

## Vorbereitung der Arbeitsumgebung

In einem ersten Schritt definierne wir unser Arbeitsverzeichnis. 

In [ ]:
import os
os.getcwd()
os.chdir("c:/Users/mhu/Documents/gitHub/magpie_chatbot")
# Pfad Privatrechner
# os.chdir("c:/Users/Hueck/OneDrive/Dokumente/GitHub/magpie_langchain")

Lade Umgebungsvariablen (inkl. OpenAI-API-Key) und initialisiere den Chatbot mit dem Modell "gpt-4o" von OpenAI.

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
llm = ChatOpenAI(model="gpt-4o")

  Im folgenden stellen wir Verbindung zur Magpie her. Wir schauen uns dan alle in der Magpie befindlichen Tabellen an. Schließlich wählen wir `view_daten_reichweite_menge` aus und speichern diese als Pandas Data Frame zur einfachen exploration der Datentabelle.

In [ ]:
import pandas as pd
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit 

db = SQLDatabase.from_uri("duckdb:///data/view_magpie.db") 

db.run("SHOW TABLES")

query = "SELECT * FROM view_daten_reichweite_menge;"
df = pd.read_sql(query, db._engine)
df

## Tools

### Standardisierte Langchain Tools

Wir intitalsieren ein standardisiertes Toolkit. Es stellt Funktionen bereit, um SQL-Queries über natürliche Sprache zu erzeugen und auszuführen. Wir lassen uns Namen und Funktion der standardisierten Tools anzeigen:

In [5]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for tool in tools:
    print(f"Tool Name: {tool.name}")
    print(f"Description: {tool.description}")
    print("-" * 40)

Tool Name: sql_db_query
Description: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
----------------------------------------
Tool Name: sql_db_schema
Description: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3
----------------------------------------
Tool Name: sql_db_list_tables
Description: Input is an empty string, output is a comma-separated list of tables in the database.
----------------------------------------
Tool Name: sql_db_query_checker
Description: Use this tool to double check if your query is

Die Standardtools aus`SQLDatabaseToolkit`können also die folgenden Standardfunktionen übernehmen: 

- `sql_db_query`  
  Führt eine übergebene SQL-Abfrage aus. Gibt das Ergebnis oder eine Fehlermeldung zurück. Bei Fehlern wie „Unknown column“ sollte zuvor das Tabellenschema geprüft werden.

- `sql_db_schema`  
  Gibt das Schema (Spaltennamen und -typen) sowie Beispielzeilen für angegebene Tabellen zurück. Vorher sollte geprüft werden, ob die Tabellen existieren.

- `sql_db_list_tables`  
  Listet alle Tabellen in der verbundenen Datenbank auf.

- `sql_db_query_checker`  
  Prüft eine SQL-Abfrage auf syntaktische Korrektheit, bevor sie mit sql_db_query ausgeführt wird. Sollte immer vorher verwendet werden.


### Maßgeschneiderte Langchain Tools

#### Retriever `rt_beschr_variable`

`rt_beschr_variable` erlaubt die semantischen Suche über Werte aus einer Datenbankspalte. 

1. Wir sammeln sämtliche Unique Werte aus `beschr_variable` und wandeln diese mit OpenAIs Embeddings-Methdode `text-embedding-3-large` in Embeddings um. Die werden in einen Vektorstore gesichert.
2. Der Vektorstore wird in einen Retriever umgewandelt, der bei einer Anfrage die 5 ähnlichsten Begriffe zurückgibt.

Schließlich wird mit create_retriever_tool ein Tool erzeugt, das über eine Beschreibung verfügt und den Retriever kapselt. Dieses Tool kann z. B. von einem Agenten genutzt werden, um Benutzereingaben mit unsicherer Schreibweise oder unvollständigen Begriffen mit den tatsächlichen Werten in der Datenbank abzugleichen.

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.agents.agent_toolkits import create_retriever_tool
import ast
import re

##################################################################
# Generiere `rt_beschr_variable`
##################################################################

def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


beschr_variable = query_as_list(db, "SELECT variable_beschr FROM view_daten_reichweite_menge")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = InMemoryVectorStore(embeddings)
_ = vector_store.add_texts(beschr_variable)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

description = (
    "Verwenden, um Werte für Filterabfragen nachzuschlagen. Die Eingabe ist eine ungefähre Schreibweise "
    "eines Eigennamens, die Ausgabe sind gültige Eigennamen. Verwende den Begriff, der der Eingabe am ähnlichsten ist."
)

rt_beschr_variable = create_retriever_tool(
    retriever,
    name="rt_beschr_variable",
    description=description,
)

In [10]:
result = retriever.invoke("Wie hoch war die Anzahl von Studierenden ohne Abitur 2006?")
result

[Document(id='6f761e93-562b-4550-987a-1c0639b84519', metadata={}, page_content='Anzahl der Studienanfänger ohne Abitur'),
 Document(id='816b2c9f-b404-4af9-a060-b8976ba161dc', metadata={}, page_content='Anteil der Studienanfänger ohne Abitur'),
 Document(id='4f150b38-c01d-44f5-b3e5-603a63abf798', metadata={}, page_content='Anteil der Studienabsolventen ohne Abitur'),
 Document(id='72199e28-79cd-4f21-a05c-926b01143a5e', metadata={}, page_content='Studienabsolventen ohne Abitur'),
 Document(id='07f053bf-d0b9-4f27-97d4-a9bb3a6b5309', metadata={}, page_content='Anteil der Bildungsausländer (Studienabsolventen)')]

In [ ]:


##################################################################
# Generiere `rt_reichweite_variable`
##################################################################

reichweite_variable = query_as_list(db, "SELECT reichweite_beschr_list FROM view_daten_reichweite_menge")

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = InMemoryVectorStore(embeddings)

_ = vector_store.add_texts(reichweite_variable)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

description = (
    "Use to look up values to filter on. Input is an approximate spelling "
    "of the proper noun, output is valid proper nouns. Use the noun most "
    "similar to the search."
)

rt_reichweite_variable = create_retriever_tool(
    retriever,
    name="rt_reichweite_variable",
    description=description,
)

##################################################################
# Generiere `rt_beschr_wert_einheit`
##################################################################

werteinheit_variable = query_as_list(db, "SELECT wert_einheit FROM view_daten_reichweite_menge")

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = InMemoryVectorStore(embeddings)

_ = vector_store.add_texts(werteinheit_variable)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

description = (
    "Use to look up values to filter on. Input is an approximate spelling "
    "of the proper noun, output is valid proper nouns. Use the noun most "
    "similar to the search."
)

rt_werteinheit_variable = create_retriever_tool(
    retriever,
    name="rt_beschr_wert_einheit",
    description=description,
)

In [ ]:
# Beispielabfrage an den Retriever
query = "Wie viele Studienanfänger gab es 2019 in Deutschland?"  # Ersetze dies durch einen tatsächlichen Suchbegriff
results = rt_reichweite_variable.run(query)
print(results)
 

In [ ]:
from langchain_core.tools import tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

@tool
def variable_beschr(user_question: str) -> str:
    """
    Sucht nach variable_beschr in Tabelle
    """
    # GANZE Frage an den Retriever geben, nicht nur einen Ausschnitt
    matches = rt_beschr_variable.invoke(user_question)
    print("ergebnisse rt_beschr_variable:", matches)
    if not matches:
        return "Error: Keine passende Variable gefunden."

    best_match = matches.split("\n")[0]  # Relevantes Ergebnis
    query = f"SELECT variable_beschr FROM view_daten_reichweite_menge WHERE variable_beschr = '{best_match}' LIMIT 1;"
    print(f"SQL Query (Variable): {query}")  # <<< HIER!
    result = db.run_no_throw(query)

    return result if result else "Error: Keine passende Variable gefunden."


@tool
def get_reichweite_beschr_list(user_question: str) -> str:
    """
    Sucht nach reichweite_beschr_list in Tabelle
    """
    # Verwende das bestehende Retriever-Tool, um die beste Übereinstimmung zu finden
    matches = rt_reichweite_variable.invoke(user_question)
    print("ergebnisse get_reichweite_beschr_list:", matches)
    if not matches:
        return "Error: Keine passende Variable gefunden."

    prompt_template = """
    Gegeben ist eine Liste von Reichweiten:
    {matches}

    Welche Reichweite passt am besten zu dieser Frage: "{user_question}"?
    Gib mir nichts als die entsprechende Reichweite zurück.
    """

    prompt = PromptTemplate(input_variables=["matches", "user_question"], template=prompt_template)
    
    # Verwende RunnableSequence anstelle von LLMChain
    chain = prompt | llm
    result = chain.invoke(input={"matches": "\n".join(matches), "user_question": user_question})
    print("Ergebniss CHatgptabfrage:", result.content)

    # SQL-Abfrage mit der gefundenen besten Übereinstimmung
    query = f"SELECT reichweite_beschr_list FROM view_daten_reichweite_menge WHERE reichweite_beschr_list = '{result.content}' LIMIT 1;"
    print(f"SQL Query (Reichweite): {query}")  # <<< HIER!
    result = db.run_no_throw(query)

    return result if result else "Error: Keine passende Variable gefunden."

# @tool
# def get_wert_einheit_id(user_question: str) -> str:
#     """Sucht in der Tabelle 'wert_einheit' nach einer passenden ID basierend auf der Beschreibung."""
#     query = f"SELECT id FROM wert_einheit WHERE beschr LIKE '%{description}%' LIMIT 1;"
#     result = db.run_no_throw(query)
#     return result if result else "Error: Keine passende Werteinheit gefunden."

 


tools.extend([variable_beschr, get_reichweite_beschr_list])

In [ ]:
test_output = variable_beschr.invoke("Wie viele Drittmittel von Stiftungen wurden 2020 in Deutschland ausgeschüttet")

print("Ergebnis:", test_output)

In [ ]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1, "Die Anzahl der Nachrichten im Template ist nicht 1!"
# Bearbeite die bestehende Nachricht, indem du Text hinzufügst
prompt_template.messages[0].prompt.template += (
    "\nYou are Sparklehorse, a chatbot for the Stifterverband organization. "
    "Your primary task is to answer questions related to the Magpie database."
)

prompt_template.messages[0].pretty_print()

In [ ]:
system_message = prompt_template.format(
    dialect=db.dialect, 
    top_k=5
)

print(system_message)

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

# Systemnachricht mit extra Anweisungen
suffix = (
    "Bevor eine SQL-Abfrage generiert wird, beachte folgendes:\n"
    "1. Benutze in Deinen Antworten immer der Werte von variable_beschr bzw get_reichweite_beschr_list, die du in deiner SQL abfrage verwendet hast, um irrtmümer zu vermeiden\n"
    "2. Verwende immer 'variable_beschr', um die korrekte Variablenbeschreibung basierend auf der Nutzeranfrage zu ermitteln und damit 'variable_beschr' zu filtern.\n"
    "3. Verwende immer 'get_reichweite_beschr_list', um die korrekte Reichweitenbeschreibung zu erhalten und damit 'get_reichweite_beschr_list' zu filtern.\n"
    "4. Um die Daten abzurufen, verwende immer die tabelle view_daten_reichweite_menge.\n"
    "5. Achte darauf, dass, wenn nach Daten zu Deutschland gefragt wird, Deutschland allein reichweite_beschr_list steht, also keine | oder anderen Werte\n"
    "6. Gib immer die finale SQL-Abfrage aus und erkläre sie. Rate niemals einen Wert oder eine ID — diese müssen immer mit den bereitgestellten Tools abgefragt werden.\n"
    "7. Falls eine ID nicht gefunden werden kann, weise darauf hin und erkläre, was fehlt.\n"
    "8. Filter Zeiträume auf diese Weise: date_part('year', zeit_start) = 2020\n"
    "9. Beachte auch immer die Variable wert_einheit: Sie gibt Informationen über Bedeutung der Werte: Das sind möglich Ausprägungen: in Tsd. Euro, Anzahl, Prozent, VZÄ, Mitarbeiter" 
)

# wert_einheit
# in Tsd. Euro    158051
# Anzahl           88177
# Prozent          11387
# VZÄ               4078
# Mitarbeiter       1380
# Mittelwert        1379
# Faktorlevel        262
# Punkte              70
# Name: count, dtype: int64


system = f"{system_message}\n\n{suffix}"

# Neuen ReAct-Agent erstellen mit den vollständigen Tools
agent_executor = create_react_agent(llm, tools, state_modifier=system)


In [ ]:
# Testanfrage an den Agenten
question = "Wie viele Studienanfänger ohne Abitur gab es 2016?"

for step in agent_executor.stream(
    {"messages": [HumanMessage(content=question)]}, 
    stream_mode="values"
):
    step["messages"][-1].pretty_print()

In [ ]:
%%sql duckdb:///C:/Users/mhu/Documents/github/magpie_chatbot/data/view_magpie.db
SELECT * FROM view_daten_reichweite_menge LIMIT 10;


In [ ]:
%%sql duckdb:///C:/Users/mhu/Documents/github/magpie_chatbot/data/view_magpie.db
SELECT * FROM view_daten_reichweite_menge LIMIT 10;
